In [1]:
import pandas as pd
import numpy as np

# Encuestas a ingresantes

A continuación vamos a estudiar los datos que nos pasaron de encuestas que se le realizaron a ingresantes de la FCEN.

Los datasets originales NO serán subidos al repositorio, pues veremos que contienen información delicada.

La información se corresponde con datos del segundo cuatrimestre de 2023 en adelante.

## RIESGO +ACOMPAÑAMIENTO 2C 2023

El primer archivo que voy a analizar es un archivo de excel que tiene 3 hojas.

In [ ]:
# Leer el archivo Excel
xls = pd.ExcelFile("../../../assets/bronze/ENCUESTAS+LISTADOS/RIESGO +ACOMPAÑAMIENTO 2C 2023.xlsx")
hojas = xls.sheet_names
print(hojas)

### Primer hoja

In [4]:
segundo_c_2023_1 = pd.read_excel(xls, sheet_name=hojas[0])

In [ ]:
with pd.option_context('display.max_rows', None):
    print(segundo_c_2023_1.dtypes)

In [6]:
nulos = segundo_c_2023_1.isnull().sum()
nulos.to_csv('nulos_Encuesta_a_Estudiantes_Iniciale.csv')

Note que varias de las filas se estaban descargando de manera incorrecta por problemas de tipos, al descargar la página como csv se solucionó el problema:

In [7]:
df = pd.read_csv('../../../assets/silver/RIESGO +ACOMPAÑAMIENTO 2C 2023/RIESGO +ACOMPAÑAMIENTO 2C 2023.xlsx - Encuesta a Estudiantes Iniciale.csv')

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df.dtypes)

In [ ]:
nulos = df.isnull().sum()
nulos.to_csv('nulos_Encuesta_a_Estudiantes_Iniciale_csv.csv')

In [ ]:
distinct_values = df['ID de respuesta'].nunique()
print(distinct_values)

In [ ]:
frecuencia = df['ID de respuesta'].value_counts()

# Muestra los valores que aparecen más de una vez
print("Frecuencia de valores en 'Id':")
with pd.option_context('display.max_rows', None):
    print(frecuencia[frecuencia > 1])

In [ ]:
# No se sube pues se veía DNI de las personas
df[df['ID de respuesta'] == 11]

Sin embargo, solo hay 4 DNI's duplicados

In [ ]:
frecuencia_dni = df['Nº de documento'].value_counts()

# Muestra los valores que aparecen más de una vez
print("Frecuencia de valores en 'Id':")
with pd.option_context('display.max_rows', None):
    print(frecuencia_dni[frecuencia_dni > 1])

In [ ]:
df['Nº de documento'].nunique()

In [ ]:
df.shape

In [ ]:
# Convertir la columna a string (por si acaso no lo está)
df['Nº de documento'] = df['Nº de documento'].astype(str)

# Eliminar espacios al principio y al final
df['Nº de documento'] = df['Nº de documento'].str.strip()

In [ ]:
df['Nº de documento'].nunique()

Veo que hay un gran número de intersecciones con los DNI's disponibles en ENCUESTAS+LISTADOS/SEMAFORO 2C 2023.xlsx

In [ ]:
semaforo_2c_2023 = pd.read_excel('../../../assets/bronze/ENCUESTAS+LISTADOS/SEMAFORO 2C 2023.xlsx')
semaforo_2c_2023.shape

In [ ]:
with pd.option_context('display.max_rows', None):
    print(semaforo_2c_2023.dtypes)

In [ ]:
# Encuentra los valores que están en ambas columnas
# Convierte la columna a cadenas de texto
df['Nº de documento'] = df['Nº de documento'].astype(str)
semaforo_2c_2023['Nº de documento'] = semaforo_2c_2023['Nº de documento'].astype(str).str.strip()

intersection = df[df['Nº de documento'].isin(semaforo_2c_2023['Nº de documento'])]['Nº de documento']

# Elimina duplicados (si es necesario)
intersection = intersection.drop_duplicates()

print("Intersección:", intersection.tolist())

In [ ]:
intersection2 = semaforo_2c_2023[semaforo_2c_2023['Nº de documento'].isin(df['Nº de documento'])]['Nº de documento']

In [ ]:
intersection.isin(intersection2).sum()

In [ ]:
semaforo_2c_2023['Nº de documento'].nunique()

In [ ]:
semaforo_2c_2023.shape

In [ ]:
frecuencia_dni_semaforo = semaforo_2c_2023['Nº de documento'].value_counts()

# Muestra los valores que aparecen más de una vez
print("Frecuencia de valores en 'Id':")
with pd.option_context('display.max_rows', None):
    print(frecuencia_dni_semaforo[frecuencia_dni_semaforo > 1])

In [ ]:
semaforo_2c_2023[semaforo_2c_2023['Nº de documento']=='45689809']

In [ ]:
df[df['Nº de documento']=='45689809']

### Segunda hoja

Definen una grilla de puntaje:

In [ ]:
segundo_c_2023_2 = pd.read_excel(xls, sheet_name=hojas[1], header=1)
segundo_c_2023_2.head()

In [ ]:
segundo_c_2023_2.dtypes

In [ ]:
#count missing values
print(segundo_c_2023_2.isnull().sum())

Saco las filas que son completamente nulas.

In [ ]:
segundo_c_2023_2.dropna(how='all', inplace=True)
print(segundo_c_2023_2.isnull().sum())

In [ ]:
segundo_c_2023_2.reset_index(inplace=True, drop=True)
segundo_c_2023_2.shape

Hay columnas que fueron autocompletadas con formulas de excel:

* Id columna: concatenación entre "Columna" y "Valor"
* Valor: Toma "valor=puntaje" y se queda con lo que está antes del símbolo =
* Puntaje: Toma "valor=puntaje" y se queda con lo que está despues del símbolo =

In [ ]:
segundo_c_2023_2['Puntaje'].unique()

In [ ]:
segundo_c_2023_2[segundo_c_2023_2['Puntaje'].isna()]

In [ ]:
# Función para extraer el valor después del "="
def extraer_puntaje(valor):
    if isinstance(valor, str) and "=" in valor:  # Verifica si es una cadena y contiene "="
        return valor.split("=")[1].replace(",", ".")  # Extrae el valor y reemplaza coma por punto
    return valor  # Si no cumple las condiciones, déjalo igual

# Aplica la función a la columna "Valor" y crea la columna "Puntaje"
segundo_c_2023_2['Puntaje'] = segundo_c_2023_2['valor=puntaje'].apply(extraer_puntaje)
segundo_c_2023_2['Puntaje'] = pd.to_numeric(segundo_c_2023_2['Puntaje'], errors='coerce')

In [ ]:
segundo_c_2023_2['Puntaje'].unique()

In [ ]:
segundo_c_2023_2['Valor'].unique()

Me guardo el resultado en un nuevo csv

In [ ]:
hojas[1]

In [ ]:
segundo_c_2023_2.to_csv('../../../assets/silver/RIESGO +ACOMPAÑAMIENTO 2C 2023/grilla_puntaje.csv', index=False)

### Tercer hoja

In [ ]:
segundo_c_2023_3 = pd.read_excel(xls, sheet_name=hojas[2])
segundo_c_2023_3

In [ ]:
hojas[2]

In [ ]:
#lo guardo sin el indice
segundo_c_2023_3.to_csv('../../../assets/silver/RIESGO +ACOMPAÑAMIENTO 2C 2023/grilla_riesgo.csv', index=False)

In [ ]:
pd.read_csv('../../../assets/silver/RIESGO +ACOMPAÑAMIENTO 2C 2023/grilla_riesgo.csv')

## Análisis 2C2023 FISICA

In [ ]:
# Leer el archivo Excel
xls_fisica_2023 = pd.ExcelFile("../../../assets/bronze/ENCUESTAS+LISTADOS/Análisis 2C2023 FISICA.xlsx")
hojas_fisica_2023 = xls_fisica_2023.sheet_names
print(hojas_fisica_2023)

In [ ]:
segundo_c_2023_fisica = pd.read_excel(xls_fisica_2023, sheet_name=hojas_fisica_2023[0])

In [ ]:
segundo_c_2023_fisica.dtypes

In [ ]:
segundo_c_2023_fisica.shape

In [ ]:
segundo_c_2023_fisica['LU'] = segundo_c_2023_fisica['LU'].astype(str).str.strip()

In [ ]:
segundo_c_2023_fisica['LU'].nunique()